<a href="https://colab.research.google.com/github/sanketn22/ihack/blob/main/ihack_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-gpu==2.11.0

In [ ]:
import tensorflow as tf

In [ ]:
!git clone https://github.com/sanketn22/ihack.git


In [ ]:
mv /content/ihack/* /content/

In [ ]:
## "cloning tfod 2.0 git"

In [ ]:
!git clone https://github.com/tensorflow/models.git

In [ ]:
cd /content/models/research/

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!pip install tensorflow-object-detection-api

In [ ]:
#%env PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim

In [ ]:
!git clone https://github.com/cocodataset/cocoapi.git

In [ ]:
cd cocoapi/PythonAPI

In [ ]:
!make

In [ ]:
!cp -r pycocotools /content/models/research

In [ ]:
cd ..

In [ ]:
cd ..

In [ ]:
!cp object_detection/packages/tf2/setup.py .

In [ ]:
!python -m pip install .

In [ ]:
cd /content/models/research


In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
cd /content/training_demo/pre-trained-models 

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz

In [ ]:
!tar -xvf ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz

In [ ]:
cd /content/training_demo

In [ ]:
# Create train data:
!python generate_tfrecord.py -x /content/training_demo/images/train -l /content/training_demo/annotations/label_map.pbtxt -o /content/training_demo/annotations/train.record

# Create test data:
!python generate_tfrecord.py -x /content/training_demo/images/test -l /content/training_demo/annotations/label_map.pbtxt -o /content/training_demo/annotations/test.record

In [ ]:
!python model_main_tf2.py --model_dir= /content/training_demo/models/my_ssd_resnet101_v1_fpn --pipeline_config_path=/content/training_demo/models/my_ssd_resnet101_v1_fpn/pipeline.config

In [ ]:
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path /content/training_demo/models/my_ssd_resnet101_v1_fpn/pipeline.config --trained_checkpoint_dir /content/training_demo --output_directory /content/training_demo/exported_models/my_model

In [ ]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
from object_detection.utils import label_map_util
import tensorflow as tf
import os
%matplotlib inline

In [ ]:
pwd

## Inference code

In [ ]:
"""
Object Detection (On Image) From TF2 Saved Model
=====================================
"""

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf
import cv2
import argparse
from google.colab.patches import cv2_imshow

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# PROVIDE PATH TO IMAGE DIRECTORY
IMAGE_PATHS = '/content/training_demo/images/test/pan_from_internet.png'


# PROVIDE PATH TO MODEL DIRECTORY
PATH_TO_MODEL_DIR = '/content/training_demo/exported_models/my_model'

# PROVIDE PATH TO LABEL MAP
PATH_TO_LABELS = '/content/training_demo/annotations/label_map.pbtxt'

# PROVIDE THE MINIMUM CONFIDENCE THRESHOLD
MIN_CONF_THRESH = float(0.60)

# LOAD THE MODEL

import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL = "/content/training_demo/exported_models/my_model" + "/saved_model"

print('Loading model...', end='')
start_time = time.time()

# LOAD SAVED MODEL AND BUILD DETECTION FUNCTION
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

# LOAD LABEL MAP DATA FOR PLOTTING

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.
    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.
    Args:
      path: the file path to the image
    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))




print('Running inference for {}... '.format(IMAGE_PATHS), end='')

image = cv2.imread(IMAGE_PATHS)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_expanded = np.expand_dims(image_rgb, axis=0)

# The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
input_tensor = tf.convert_to_tensor(image)
# The model expects a batch of images, so add an axis with `tf.newaxis`.
input_tensor = input_tensor[tf.newaxis, ...]

# input_tensor = np.expand_dims(image_np, 0)
detections = detect_fn(input_tensor)

# All outputs are batches tensors.
# Convert to numpy arrays, and take index [0] to remove the batch dimension.
# We're only interested in the first num_detections.
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
               for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

image_with_detections = image.copy()

# SET MIN_SCORE_THRESH BASED ON YOU MINIMUM THRESHOLD FOR DETECTIONS
viz_utils.visualize_boxes_and_labels_on_image_array(
      image_with_detections,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=0.5,
      agnostic_mode=False)

print('Done')
# DISPLAYS OUTPUT IMAGE
cv2_imshow(image_with_detections)
# CLOSES WINDOW ONCE KEY IS PRESSED


## **EasyOcr**

# New Section

In [ ]:
pwd


'/content/training_demo'

In [ ]:
cd ..


/content


In [ ]:
cd /content/models/research


[Errno 2] No such file or directory: '/content/models/research'
/content/training_demo


In [ ]:
!pip install easyocr
!pip install opencv-python

In [ ]:
!git clone https://github.com/sanketn22/ihack.git

Cloning into 'ihack'...
fatal: unable to access 'https://github.com/sanketn22/ihack.git/': The requested URL returned error: 503


In [ ]:
pwd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import easyocr
from pylab import rcParams
from IPython.display import Image
from easyocr import Reader
import argparse
import cv2
import os
rcParams['figure.figsize'] = 8 , 16


In [ ]:
reader = easyocr.Reader(['en','hi'])

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

RuntimeError: ignored

In [ ]:
cd /content/training_demo


In [ ]:
im=Image.open('/content/training_demo/images/train/aadhaar-card (11).jpg')

#folder_path = '/path/to/images'

# Iterate through the images in the folder
#for file in os.listdir(folder_path):
    # Get the full path to the image
 #   file_path = os.path.join(folder_path, file)

In [ ]:
output = reader.readtext('/content/training_demo/images/train/aadhaar-card (11).jpg')

In [ ]:
output


In [ ]:
from PIL import Image , ImageDraw , ImageFont

In [ ]:
import cv2
import numpy as np
import re
top_left = tuple(output[0][0][0])
bottom_right = tuple(output[0][0][2])
text1 = output[0][1]
font = cv2.FONT_HERSHEY_SIMPLEX

NameError: ignored

In [ ]:
img = cv2.imread('/content/training_demo/images/train/aadhaar-card (11).jpg')
spacer = 300
font = cv2.FONT_HERSHEY_SIMPLEX

for detection in output:
  top_left = tuple(int(val) for val in detection[0][0])
  bottom_right = tuple(int(val) for val in detection[0][2])
  text1 = detection[1]
  img = cv2.rectangle(img,top_left,bottom_right,(0,255,255),3)
  img = cv2.putText(img,text1,bottom_right,font,0.8,(0,0,255),2,cv2.LINE_AA)
  spacer +=10

plt.figure(figsize=(10,10))
plt.imshow(img)
plt.show()

In [ ]:
f=open('/content/ihack/output2.txt', 'w', encoding='utf-8')

for t in output:
    f.write(' '.join(t[1]) + '\n')
  

In [ ]:
open("/content/ihack/output2.txt", 'r')

In [ ]:

def isValidNum(str):
    regex="^[2-9]{1}[0-9]{3}[ ][0-9]{4}[ ][0-9]{4}$"; #aadhaar card

    #regex="^[A-Z]{5}[0-9]{4}[A-Z]{1}";
 
    # Compile the ReGex
    p = re.compile(regex);
    
    if (str == ''):
        return False;

    m = re.match(p, str);
     
    return m
    # if m is None:
    #     return False
    # else:
    #     return True

for i in output:
       o = isValidNum(i[1])
       if(o != None):
          print(i[1])

    
  

In [ ]:
def FindAadhaar(txt):
    my_file = open(txt, "r")
    data = my_file.read()
    data= data.split()
    a=[]
    for i in range(len(data)-2):
        if (len(data[i])==4 and len(data[i+1])==4 and len(data[i+2])==4):
            a.append(data[i]+" "+data[i+1]+" "+data[i+2])
        if (len(data[i])==8 and len(data[i+1])==4):
            a.append(data[i]+" "+data[i+1])
        if (len(data[i])==12):
            a.append(data[i])

    for i in a:
        if "VID" in i:
            a.remove(i)
#AADHAAR NUMBER NEVER STARTS FROM 0 OR 1
    for i in a:
        if i[0]=='0'or'1':
            a.remove(i)
        
    replace={"B":"8","Z":"2","O":"0","I":"1","l":"1"," ":""}
    for i in range(len(a)):
        for j in replace.keys(): 
            a[i]=a[i].replace(j,replace[j])

    for i in a:
        if any(c.isalpha() for c in i)==True:
            a.remove(i)

#verhoeff algorithm
    mult = [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 0, 6, 7, 8, 9, 5], [2, 3, 4, 0, 1, 7, 8, 9, 5, 6],
            [3, 4, 0, 1, 2, 8, 9, 5, 6, 7], [4, 0, 1, 2, 3, 9, 5, 6, 7, 8], [5, 9, 8, 7, 6, 0, 4, 3, 2, 1],
            [6, 5, 9, 8, 7, 1, 0, 4, 3, 2], [7, 6, 5, 9, 8, 2, 1, 0, 4, 3], [8, 7, 6, 5, 9, 3, 2, 1, 0, 4],
            [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]]
    perm = [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 5, 7, 6, 2, 8, 3, 0, 9, 4], [5, 8, 0, 3, 7, 9, 6, 1, 4, 2],
            [8, 9, 1, 6, 0, 4, 3, 5, 2, 7], [9, 4, 5, 3, 1, 2, 6, 8, 7, 0], [4, 2, 8, 6, 5, 7, 3, 9, 0, 1],
            [2, 7, 9, 3, 8, 0, 6, 4, 1, 5], [7, 0, 4, 6, 9, 1, 3, 2, 5, 8]]

    def Validate(aadharNum):
        try:
            i = len(aadharNum)
            j = 0
            x = 0

            while i > 0:
                i -= 1
                x = mult[x][perm[(j % 8)][int(aadharNum[i])]]
                j += 1
            if x == 0:
                return 1
            else:
                return 0

        except ValueError:
            return 'Invalid Aadhaar Number'
        except IndexError:
            return 'Invalid Aadhaar Number'
    
    count=0
    for i in a:
        if Validate(i)==1:
            return(print("Valid Aadhaar Number found is: "+i[0:4]+" "+i[4:8]+" "+i[8:]))
            count=1
    if count==0:
        return(print("No Valid Aadhaar Number"))
            
#TESTING FUNCTION
FindAadhaar("/content/ihack/output2.txt")

tesseract

In [ ]:
cd /content/models/research

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

In [ ]:
import pytesseract
import shutil
import os
import random
try:
 from PIL import Image
except ImportError:
 import Image

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import cv2
from PIL import Image
import pytesseract
import argparse
import os

image = cv2.imread("/content/training demo/images/train/pan2.png")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
filename = "{}.png".format(os.getpid())
kernel = np.ones((1, 1), np.uint8)
img = cv2.dilate(img, kernel, iterations=1)
img = cv2.erode(img, kernel, iterations=1)
cv2.imwrite(filename,gray)

text = pytesseract.image_to_string(Image.open(filename))
print(text)
os.remove(filename)

check whether pancard no is valid

In [ ]:
def isValidPan(pancard_no):
  if (pancard_no == ''):
      return False;
  regex = pancard_no[0]+ "[" + pancard_no[0] + "-Z][" +pancard_no[1] + "-Z]" + "[PHATFC]" + "[A-Z]"+"[0-9]{3}[1-9]{1}[A-Z0-9]"
  p = re.compile(regex);
  m = re.match(p, pancard_no);
     
  if m is None:
      return False
  else:
      return True


extract info from pancard

In [ ]:
def extract_info(text,type_of_doc):
  if type_of_doc == "pancard":
    regex = ["(?:Number|Number Card)\s*([a-z]{5}\d{4}[a-z]{1})", "(?:Number|Number Card)\s*([a-zA-Z0–9 ]{8,13})","([a-z]{5}\d{4}[a-z]{1})", "([a-z]{5}\d{1,4}[a-z]{1,3})"]

